In [15]:
import cv2 as cv
import numpy as np
import math
import tensorflow as tf
import os
from IPython.display import clear_output




In [16]:
def display(img):
    cv.namedWindow('img', cv.WINDOW_NORMAL)
    cv.imshow('img', img)
    cv.waitKey(0)
    cv.destroyAllWindows()

def resize(img,scale = 0.3):
    width = int(img.shape[1]*scale)
    height = int(img.shape[0]*scale)
    dim = (width,height)
    return cv.resize(img,dim,interpolation = cv.INTER_AREA)

def distance(cord1,cord2):
   cord1= np.array(cord1)
   cord2 = np.array(cord2)
   return math.sqrt(np.sum((cord1-cord2)**2))

img = cv.imread("/home/abhijith/Desktop/Active/Hand-writing-rep/test.jpg")
img = resize(img)

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
edged = cv.Canny(gray, 50, 200)

threshold_value = 150
_, binary_image = cv.threshold(gray, threshold_value, 255, cv.THRESH_BINARY)
# display(binary_image)
inputBINARY =cv.bitwise_not(binary_image)
inputBINARY = cv.GaussianBlur(inputBINARY, (5, 5), 0)
contours,_ = cv.findContours(edged,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_NONE)

cords =[]
test = img.copy()
for cont in contours:
    x,y,w,h = cv.boundingRect(cont)
    cords.append([x,y])
cords = np.array(cords)

Dthresh = 30

distanceMatrix=[]

for cor1 in cords:

    temp=[]
    for cor2 in cords:
        dist = distance(cord1=cor1,cord2=cor2)
        if dist <= Dthresh:
            temp.append(1)
        else:
            temp.append(0)
        
        
    distanceMatrix.append(temp)

distanceMatrix = np.array(distanceMatrix)
visited = []  # stores indexes
connected = []  # stores array of connected index
collector = []  # stores indexes

def findconnected(submatrix,idx):

    if idx not in visited:

        visited.append(idx)

        if idx not in collector:
            collector.append(idx)

        for i in range(len(submatrix)):

            if submatrix[i]==1:
                if i not in visited:
                    collector.append(i)
        
        for nodeidx in collector:
            if nodeidx not in visited:
                findconnected(distanceMatrix[nodeidx],nodeidx)

for runidx in range(len(distanceMatrix)):

    findconnected(distanceMatrix[runidx],runidx)
    
    if len(collector)!=0:
        connected.append(collector)
    collector=[]

def padding(image):
    
    original_rows, original_cols = image.shape
    padding_rows = 10  
    padding_cols = 10  
    padded_rows = original_rows + 2 * padding_rows
    padded_cols = original_cols + 2 * padding_cols
    padded_image = np.zeros((padded_rows, padded_cols), dtype=np.uint8)
    padded_image[padding_rows:padding_rows + original_rows, padding_cols:padding_cols + original_cols] = image
    image = cv.resize(padded_image,(28,28))
    return image


In [17]:
alphabets = [chr(y) for y in range(ord("A"),ord("Z")+1) ]
from multiprocessing import Process, Queue


def loadedModels(path):
    models =[]
    for i in range(len(os.listdir(path))):
        model = tf.keras.models.load_model(f'{path}/{alphabets[i]}.h5')
        models.append(model)
    return models
        

def OVApipeline(models,model_Inp,conf_thresh):
    result = np.zeros(len(models))
    for i in range(len(models)):

        res = models[i].predict(model_Inp,verbose =0)
        
        result[i] = res[0][0]
        # if res[0][0]>conf_thresh:
        #     break
    return result

def analysis(image):
    
    model_Inp = image.reshape(1,28,28)
    result =OVApipeline(models,model_Inp,conf_thresh=0.7)
    
    index = np.argmax(result)

    alph = "undef"
    if(np.max(result)>0.8):
        alph = alphabets[index]
    
    # print("/////////////////////////",alph) 
    # print(result[index])
   
    return index

In [18]:

path = "/home/abhijith/Desktop/Active/Hand-writing-rep/emnist-47-unBlanced-Isolated/ISO-MERGE"
models = loadedModels(path)

In [ ]:
ALP_arr = []
Dimn_arr =[]

valiating = alphabets[:]
for i in range(26):
    ALP_arr.append([])
    Dimn_arr.append([])

Similar_contour_cluster = []
similar_contour_dimen = []

processes =[]

for con in connected:
    temp = np.array([])

    for idx in con:
        if len(temp)==0:
            temp = contours[idx]
        else:
            temp = np.concatenate((temp,contours[idx]))
    
    x,y,w,h = cv.boundingRect(temp)
    
    if w>12:

        Similar_contour_cluster.append(temp)
        similar_contour_dimen.append([x,y,w,h])
        
        image = inputBINARY[y:y+h,x:x+w]
        image = padding(image)   
        res = analysis(image)
        
        valiating[res]=99
        # clear_output(wait=True)

valid =  [i for i in valiating if i != 99]

       


In [ ]:

print("missing",valid)
# print(valiating)

missing ['G']


In [ ]:
print(chr(48))

0
